In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [3]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [4]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [5]:
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [8]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    print(grid_df.info())
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    X_valid=grid_df[valid_mask][features_columns]
    
    valid_pred = grid_df[valid_mask].reset_index(drop=True)
    cols = [col for col in ['id', 'd']]
    valid_pred = valid_pred[cols]
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    valid_pred['pred']=estimator.predict(X_valid)
    valid_pred.to_pickle('valid_pred_'+store_id+'.pkl')
    print(valid_pred.info())
    del valid_pred
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 

[100]	valid_0's rmse: 1.89284
[200]	valid_0's rmse: 1.85131
[300]	valid_0's rmse: 1.83969
[400]	valid_0's rmse: 1.83187
[500]	valid_0's rmse: 1.82598
[600]	valid_0's rmse: 1.82106
[700]	valid_0's rmse: 1.8151
[800]	valid_0's rmse: 1.8101
[900]	valid_0's rmse: 1.80468
[1000]	valid_0's rmse: 1.80055
[1100]	valid_0's rmse: 1.79617
[1200]	valid_0's rmse: 1.79187
[1300]	valid_0's rmse: 1.78823
[1400]	valid_0's rmse: 1.78449
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85372 entries, 0 to 85371
Data columns (total 3 columns):
id      85372 non-null category
d       85372 non-null int16
pred    85372 non-null float64
dtypes: category(1), float64(1), int16(1)
memory usage: 1.2 MB
None
Train CA_3
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_T

[100]	valid_0's rmse: 1.3326
[200]	valid_0's rmse: 1.32555
[300]	valid_0's rmse: 1.32091
[400]	valid_0's rmse: 1.31693
[500]	valid_0's rmse: 1.3135
[600]	valid_0's rmse: 1.31045
[700]	valid_0's rmse: 1.30736
[800]	valid_0's rmse: 1.30489
[900]	valid_0's rmse: 1.30197
[1000]	valid_0's rmse: 1.29876
[1100]	valid_0's rmse: 1.29612
[1200]	valid_0's rmse: 1.29381
[1300]	valid_0's rmse: 1.29141
[1400]	valid_0's rmse: 1.28865
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85372 entries, 0 to 85371
Data columns (total 3 columns):
id      85372 non-null category
d       85372 non-null int16
pred    85372 non-null float64
dtypes: category(1), float64(1), int16(1)
memory usage: 1.2 MB
None
Train TX_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_T

[100]	valid_0's rmse: 1.71833
[200]	valid_0's rmse: 1.69794
[300]	valid_0's rmse: 1.68458
[400]	valid_0's rmse: 1.67864
[500]	valid_0's rmse: 1.67315
[600]	valid_0's rmse: 1.66851
[700]	valid_0's rmse: 1.66399
[800]	valid_0's rmse: 1.66003
[900]	valid_0's rmse: 1.65663
[1000]	valid_0's rmse: 1.65273
[1100]	valid_0's rmse: 1.64889
[1200]	valid_0's rmse: 1.6459
[1300]	valid_0's rmse: 1.64224
[1400]	valid_0's rmse: 1.63887
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85372 entries, 0 to 85371
Data columns (total 3 columns):
id      85372 non-null category
d       85372 non-null int16
pred    85372 non-null float64
dtypes: category(1), float64(1), int16(1)
memory usage: 1.2 MB
None
Train TX_3
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_

[100]	valid_0's rmse: 1.59878
[200]	valid_0's rmse: 1.57937
[300]	valid_0's rmse: 1.5718
[400]	valid_0's rmse: 1.5666
[500]	valid_0's rmse: 1.56201
[600]	valid_0's rmse: 1.55715
[700]	valid_0's rmse: 1.55287
[800]	valid_0's rmse: 1.54936
[900]	valid_0's rmse: 1.54571
[1000]	valid_0's rmse: 1.54212
[1100]	valid_0's rmse: 1.53796
[1200]	valid_0's rmse: 1.53458
[1300]	valid_0's rmse: 1.53124
[1400]	valid_0's rmse: 1.52818
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85372 entries, 0 to 85371
Data columns (total 3 columns):
id      85372 non-null category
d       85372 non-null int16
pred    85372 non-null float64
dtypes: category(1), float64(1), int16(1)
memory usage: 1.2 MB
None
Train WI_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_T

[100]	valid_0's rmse: 1.92146
[200]	valid_0's rmse: 1.85492
[300]	valid_0's rmse: 1.83253
[400]	valid_0's rmse: 1.82095
[500]	valid_0's rmse: 1.81358
[600]	valid_0's rmse: 1.80564
[700]	valid_0's rmse: 1.79827
[800]	valid_0's rmse: 1.79192
[900]	valid_0's rmse: 1.7861
[1000]	valid_0's rmse: 1.78047
[1100]	valid_0's rmse: 1.77421
[1200]	valid_0's rmse: 1.76966
[1300]	valid_0's rmse: 1.76475
[1400]	valid_0's rmse: 1.76026
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85372 entries, 0 to 85371
Data columns (total 3 columns):
id      85372 non-null category
d       85372 non-null int16
pred    85372 non-null float64
dtypes: category(1), float64(1), int16(1)
memory usage: 1.2 MB
None


In [9]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  0.68 min round |  0.68 min total |  37308.80 day sales |
Predict | Day: 2
##########  0.75 min round |  1.44 min total |  35335.42 day sales |
Predict | Day: 3
##########  0.73 min round |  2.17 min total |  34783.90 day sales |
Predict | Day: 4
##########  0.74 min round |  2.91 min total |  35285.85 day sales |
Predict | Day: 5
##########  0.81 min round |  3.73 min total |  41724.47 day sales |
Predict | Day: 6
##########  0.78 min round |  4.50 min total |  50966.54 day sales |
Predict | Day: 7
##########  0.86 min round |  5.37 min total |  53580.33 day sales |
Predict | Day: 8
##########  0.88 min round |  6.25 min total |  44119.60 day sales |
Predict | Day: 9
##########  0.89 min round |  7.14 min total |  44431.43 day sales |
Predict | Day: 10
##########  0.86 min round |  8.00 min total |  38864.02 day sales |
Predict | Day: 11
##########  0.90 min round |  8.90 min total |  40720.81 day sales |
Predict | Day: 12
##########  0.93 min round |  9.82

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.858353,0.786773,0.787750,0.766156,0.987730,1.162198,1.186359,0.816469,0.941797,...,0.881215,1.094018,1.086345,0.901821,0.872095,0.850221,0.964878,1.028725,1.213543,1.079085
1,HOBBIES_1_002_CA_1_validation,0.179499,0.174155,0.171382,0.196262,0.223260,0.298377,0.328949,0.223760,0.224478,...,0.222005,0.292770,0.292519,0.207134,0.194770,0.195902,0.197496,0.219907,0.301947,0.284174
2,HOBBIES_1_003_CA_1_validation,0.397293,0.406841,0.433615,0.422570,0.572800,0.700690,0.616532,0.481706,0.480695,...,0.521024,0.649324,0.637690,0.477059,0.439545,0.420434,0.453030,0.581819,0.683485,0.667797
3,HOBBIES_1_004_CA_1_validation,1.679338,1.327381,1.385072,1.615031,1.921572,3.031844,3.416930,1.632780,1.397898,...,1.876863,2.601219,3.372410,1.747331,1.430922,1.422631,1.452189,2.027651,3.046427,3.546514
4,HOBBIES_1_005_CA_1_validation,0.843944,0.782973,0.807119,0.861665,1.058857,1.456068,1.371624,0.947247,0.893115,...,1.078430,1.515986,1.595069,1.032757,0.818845,0.927407,0.860789,1.146131,1.587761,1.463802


In [10]:
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv('sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False) #0.47388

In [ ]:
#calculate wrmsse of valid dataset

In [12]:
ca_1_pred=pd.read_pickle('valid_pred_CA_1.pkl')
print(ca_1_pred.shape)
ca_2_pred=pd.read_pickle('valid_pred_CA_2.pkl')
print(ca_2_pred.shape)
ca_3_pred=pd.read_pickle('valid_pred_CA_3.pkl')
print(ca_3_pred.shape)
ca_4_pred=pd.read_pickle('valid_pred_CA_4.pkl')
print(ca_4_pred.shape)
tx_1_pred=pd.read_pickle('valid_pred_TX_1.pkl')
print(tx_1_pred.shape)
tx_2_pred=pd.read_pickle('valid_pred_TX_2.pkl')
print(tx_2_pred.shape)
tx_3_pred=pd.read_pickle('valid_pred_TX_3.pkl')
print(tx_3_pred.shape)
wi_1_pred=pd.read_pickle('valid_pred_WI_1.pkl')
print(wi_1_pred.shape)
wi_2_pred=pd.read_pickle('valid_pred_WI_2.pkl')
print(wi_2_pred.shape)
wi_3_pred=pd.read_pickle('valid_pred_WI_3.pkl')
print(wi_3_pred.shape)

(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)
(85372, 3)


In [13]:
ca_1_pred.head()

,id,d,pred
0,HOBBIES_1_001_CA_1_validation,1886,0.879609
1,HOBBIES_1_002_CA_1_validation,1886,0.187493
2,HOBBIES_1_003_CA_1_validation,1886,0.357378
3,HOBBIES_1_004_CA_1_validation,1886,1.735800
4,HOBBIES_1_005_CA_1_validation,1886,0.973022


In [14]:
valid_pred=pd.concat([ca_1_pred,ca_2_pred,ca_3_pred,ca_4_pred,tx_1_pred,tx_2_pred,tx_3_pred,
                      wi_1_pred,wi_2_pred,wi_3_pred], axis=0).reset_index(drop=True)

In [15]:
valid_pred.shape

(853720, 3)

In [16]:
valid_pred.head()

,id,d,pred
0,HOBBIES_1_001_CA_1_validation,1886,0.879609
1,HOBBIES_1_002_CA_1_validation,1886,0.187493
2,HOBBIES_1_003_CA_1_validation,1886,0.357378
3,HOBBIES_1_004_CA_1_validation,1886,1.735800
4,HOBBIES_1_005_CA_1_validation,1886,0.973022


In [17]:
grid_df=pd.read_pickle('df_full.pkl')
grid_df.columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'release', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
       'price_momentum', 'price_momentum_m', 'price_momentum_y',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y',
       'tm_wm', 'tm_dw', 'tm_w_end', 'id_1', 'd_1', 'enc_state_id_mean',
       'enc_state_id_std', 'enc_store_id_mean', 'enc_store_id_std',
       'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean',
       'enc_dept_id_std', 'enc_state_id_cat_id_mean',
       'enc_state_id_cat_id_std', 'enc_state_id_dept_id_mean',
       'enc_state_id_dept_id_std', 'enc_store_id_cat_id_mean',
       'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean',
       'enc_store_id_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std',
       'enc_item_id_state_id_mean', 'enc_item_id_state_id_std',
       'enc_item_id_store_id_mean', 'enc_item_id_store_id_std', 'sales_lag_28',
       'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32',
       'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
       'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40',
       'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30',
       'rolling_std_30', 'rolling_mean_60', 'rolling_std_60',
       'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7',
       'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30',
       'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7',
       'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30',
       'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7',
       'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30',
       'rolling_mean_tmp_14_60']
grid_df.drop(grid_df.columns[34:36], axis=1,inplace=True)

In [18]:
train_mask = grid_df['d']<=END_TRAIN
valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))

In [ ]:
#valid true sale value

In [19]:
valid=grid_df[valid_mask].reset_index(drop=True)
valid=valid[['id','d', 'sales']]

In [20]:
del grid_df
gc.collect()

103

In [21]:
valid.shape

(853720, 3)

In [22]:
valid.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1886,1.0
1,HOBBIES_1_002_CA_1_validation,1886,1.0
2,HOBBIES_1_003_CA_1_validation,1886,0.0
3,HOBBIES_1_004_CA_1_validation,1886,0.0
4,HOBBIES_1_005_CA_1_validation,1886,1.0


In [23]:
valid_pred.shape

(853720, 3)

In [24]:
valid_pred.head()

,id,d,pred
0,HOBBIES_1_001_CA_1_validation,1886,0.879609
1,HOBBIES_1_002_CA_1_validation,1886,0.187493
2,HOBBIES_1_003_CA_1_validation,1886,0.357378
3,HOBBIES_1_004_CA_1_validation,1886,1.735800
4,HOBBIES_1_005_CA_1_validation,1886,0.973022


In [25]:
valid_full=pd.merge(valid, valid_pred, how='left', on=['id', 'd'])

In [26]:
valid_full.head()

,id,d,sales,pred
0,HOBBIES_1_001_CA_1_validation,1886,1.0,0.879609
1,HOBBIES_1_002_CA_1_validation,1886,1.0,0.187493
2,HOBBIES_1_003_CA_1_validation,1886,0.0,0.357378
3,HOBBIES_1_004_CA_1_validation,1886,0.0,1.735800
4,HOBBIES_1_005_CA_1_validation,1886,1.0,0.973022


In [27]:
valid_full.tail()

,id,d,sales,pred
853715,FOODS_3_823_WI_3_validation,1913,1.0,0.552181
853716,FOODS_3_824_WI_3_validation,1913,0.0,0.387571
853717,FOODS_3_825_WI_3_validation,1913,0.0,0.792679
853718,FOODS_3_826_WI_3_validation,1913,3.0,1.257791
853719,FOODS_3_827_WI_3_validation,1913,0.0,0.361039


In [29]:
valid = valid.pivot(index="id", columns="d", values="sales").reset_index()

In [30]:
valid.head()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,FOODS_1_001_CA_1_validation,2.0,1.0,1.0,0.0,4.0,0.0,0.0,4.0,1.0,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0
1,FOODS_1_001_CA_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,14.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0
2,FOODS_1_001_CA_3_validation,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,FOODS_1_001_CA_4_validation,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0
4,FOODS_1_001_TX_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
valid.tail()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
30485,HOUSEHOLD_2_516_TX_2_validation,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
30486,HOUSEHOLD_2_516_TX_3_validation,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,HOUSEHOLD_2_516_WI_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30488,HOUSEHOLD_2_516_WI_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
30489,HOUSEHOLD_2_516_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
valid.shape

(30490, 29)

In [ ]:
#valid prediction value

In [37]:
valid_pred=valid_full[['id', 'd','pred']]
valid_pred=valid_pred.pivot(index="id", columns="d", values="pred").reset_index()

In [38]:
valid_pred.head()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,FOODS_1_001_CA_1_validation,0.685397,0.651345,0.689602,0.760516,0.875410,1.078393,0.957094,0.756863,0.703181,...,0.888187,1.106833,1.063508,0.759652,0.745267,0.692920,0.768307,0.962939,1.136008,1.070437
1,FOODS_1_001_CA_2_validation,1.041330,0.976813,0.934107,0.897321,1.049483,1.311205,0.981375,0.362078,0.750064,...,0.970673,1.305012,1.454240,0.864558,0.815666,0.787203,0.929613,1.093902,1.246147,1.398081
2,FOODS_1_001_CA_3_validation,0.702838,0.701076,0.758857,0.830388,0.840845,1.354820,1.667528,0.930712,0.854505,...,1.009303,1.420445,1.837298,0.827672,0.758505,0.743554,0.796418,0.919663,1.316452,1.494654
3,FOODS_1_001_CA_4_validation,0.393409,0.357715,0.376611,0.399401,0.431834,0.451501,0.447883,0.448853,0.404760,...,0.392029,0.314133,0.319371,0.362923,0.364867,0.345212,0.404035,0.411054,0.420275,0.418220
4,FOODS_1_001_TX_1_validation,0.482882,0.470486,0.429711,0.457244,0.504416,0.391476,0.551846,0.416344,0.416020,...,0.226207,0.232233,0.258321,0.194951,0.152111,0.062426,0.062026,0.058331,0.072144,0.079430


In [39]:
valid_pred.tail()

d,id,1886,1887,1888,1889,1890,1891,1892,1893,1894,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
30485,HOUSEHOLD_2_516_TX_2_validation,0.232524,0.265407,0.276288,0.284208,0.276142,0.348009,0.332618,0.196983,0.198651,...,0.299100,0.400768,0.353148,0.190107,0.207255,0.213266,0.232036,0.348962,0.418391,0.345139
30486,HOUSEHOLD_2_516_TX_3_validation,0.103730,0.128123,0.169013,0.191267,0.213213,0.239795,0.262307,0.185408,0.181350,...,0.171682,0.208065,0.188909,0.109649,0.119730,0.125893,0.132324,0.150804,0.156234,0.130761
30487,HOUSEHOLD_2_516_WI_1_validation,0.106269,0.099141,0.099907,0.110340,0.123418,0.121354,0.132926,0.098722,0.083635,...,0.104750,0.111320,0.101076,0.071740,0.071684,0.080668,0.075995,0.111812,0.138599,0.127595
30488,HOUSEHOLD_2_516_WI_2_validation,0.047530,0.048388,0.053498,0.061219,0.067412,0.073385,0.065592,0.053000,0.050756,...,0.047732,0.053019,0.047305,0.036115,0.042968,0.039945,0.050973,0.068622,0.087753,0.077186
30489,HOUSEHOLD_2_516_WI_3_validation,0.122146,0.125954,0.114281,0.146339,0.162095,0.161025,0.140942,0.121223,0.107219,...,0.090868,0.095892,0.087782,0.066404,0.067981,0.062539,0.076859,0.100131,0.106912,0.095970


In [40]:
#calculate wrmsse
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

In [41]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [42]:
# Sales quantities:
sales = pd.read_csv('sales_train_validation.csv')

# Calendar to get week number to join sell prices:
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)

# Sell prices to calculate sales in USD:
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [43]:
# Dataframe with only last 28 days:
cols = ["d_{}".format(i) for i in range(1914-28, 1914)]
data = sales[["id", 'store_id', 'item_id'] + cols]

# To long form:
data = data.melt(id_vars=["id", 'store_id', 'item_id'], 
                 var_name="d", value_name="sale")

# Add week of year column from 'calendar':
data = pd.merge(data, calendar, how = 'left', 
                left_on = ['d'], right_on = ['d'])

data = data[["id", 'store_id', 'item_id', "sale", "d", "wm_yr_wk"]]

# Add weekly price from 'sell_prices':
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
data.drop(columns = ['wm_yr_wk'], inplace=True)

# Calculate daily sales in USD:
data['sale_usd'] = data['sale'] * data['sell_price']
data.head()

,id,store_id,item_id,sale,d,sell_price,sale_usd
0,HOBBIES_1_001_CA_1_validation,CA_1,HOBBIES_1_001,1,d_1886,8.257812,8.257812
1,HOBBIES_1_002_CA_1_validation,CA_1,HOBBIES_1_002,1,d_1886,3.970703,3.970703
2,HOBBIES_1_003_CA_1_validation,CA_1,HOBBIES_1_003,0,d_1886,2.970703,0.000000
3,HOBBIES_1_004_CA_1_validation,CA_1,HOBBIES_1_004,0,d_1886,4.640625,0.000000
4,HOBBIES_1_005_CA_1_validation,CA_1,HOBBIES_1_005,1,d_1886,2.880859,2.880859


In [44]:
# List of categories combinations for aggregations as defined in docs:
dummies_list = [sales.state_id, sales.store_id, 
                sales.cat_id, sales.dept_id, 
                sales.state_id +'_'+ sales.cat_id, sales.state_id +'_'+ sales.dept_id,
                sales.store_id +'_'+ sales.cat_id, sales.store_id +'_'+ sales.dept_id, 
                sales.item_id, sales.state_id +'_'+ sales.item_id, sales.id]

In [45]:
## First element Level_0 aggregation 'all_sales':
dummies_df_list =[pd.DataFrame(np.ones(sales.shape[0]).astype(np.int8), 
                               index=sales.index, columns=['all']).T]

In [46]:
# List of dummy dataframes:
for i, cats in enumerate(dummies_list):
    dummies_df_list +=[pd.get_dummies(cats, drop_first=False, dtype=np.int8).T]

In [47]:
# Concat dummy dataframes in one go:
## Level is constructed for free.
roll_mat_df = pd.concat(dummies_df_list, keys=list(range(12)), 
                        names=['level','id'])#.astype(np.int8, copy=False)

In [48]:
roll_mat_df.head()

0      1      2      3      4      5      6      7      8      \
level id                                                                    
0     all       1      1      1      1      1      1      1      1      1   
1     CA        1      1      1      1      1      1      1      1      1   
      TX        0      0      0      0      0      0      0      0      0   
      WI        0      0      0      0      0      0      0      0      0   
2     CA_1      1      1      1      1      1      1      1      1      1   

            9      ...  30480  30481  30482  30483  30484  30485  30486  \
level id           ...                                                    
0     all       1  ...      1      1      1      1      1      1      1   
1     CA        1  ...      0      0      0      0      0      0      0   
      TX        0  ...      0      0      0      0      0      0      0   
      WI        0  ...      1      1      1      1      1      1      1   
2     CA_1      1  ...      0      0      0      0      0      0      0   

            30487  30488  30489  
level id                         
0     all       1      1      1  
1     CA        0      0      0  
      TX        0      0      0  
      WI        1      1      1  
2     CA_1      0      0      0  

[5 rows x 30490 columns]

In [49]:
roll_mat_df.tail()

0      1      2      3      4      \
level id                                                                   
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0   

                                       5      6      7      8      9      ...  \
level id                                                                  ...   
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0  ...   
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0  ...   

                                       30480  30481  30482  30483  30484  \
level id                                                                   
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0   
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0   

                                       30485  30486  30487  30488  30489  
level id                                                                  
11    HOUSEHOLD_2_516_TX_2_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_TX_3_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_WI_1_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_WI_2_validation      0      0      0      0      0  
      HOUSEHOLD_2_516_WI_3_validation      0      0      0      0      0  

[5 rows x 30490 columns]

In [50]:
# Save values as sparse matrix & save index for future reference:
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
roll_mat_csr.shape

(42840, 30490)

In [51]:
# Dump roll matrix to pickle:
roll_mat_df.to_pickle('roll_mat_df.pkl')

In [52]:
# Free some momory:
del dummies_df_list, roll_mat_df
gc.collect()

184

In [53]:
# Fucntion to calculate S weights:
def get_s(drop_days=0):
    
    """
    drop_days: int, equals 0 by default, so S is calculated on all data.
               If equals 28, last 28 days won't be used in calculating S.
    """
    # Rollup sales:
    d_name = ['d_' + str(i+1) for i in range(1913-drop_days)]
    sales_train_val = roll_mat_csr * sales[d_name].values

    no_sales = np.cumsum(sales_train_val, axis=1) == 0
    sales_train_val = np.where(no_sales, np.nan, sales_train_val)

    # Denominator of RMSSE / RMSSE
    weight1 = np.nanmean(np.diff(sales_train_val,axis=1)**2,axis=1)
    
    return weight1

In [54]:
S = get_s(drop_days=0)

In [55]:
S

array([3.51176267e+07, 7.34126518e+06, 3.33954805e+06, ...,
       1.71293871e-01, 6.98666667e-02, 2.81004710e-01])

In [56]:
# Functinon to calculate weights:
def get_w(sale_usd):
    """
    """
    # Calculate the total sales in USD for each item id:
    total_sales_usd = sale_usd.groupby(
        ['id'], sort=False)['sale_usd'].apply(np.sum).values
    
    # Roll up total sales by ids to higher levels:
    weight2 = roll_mat_csr * total_sales_usd
    
    return 12*weight2/np.sum(weight2)

In [57]:
W = get_w(data[['id','sale_usd']])

In [58]:
W 

array([1.00000000e+00, 4.42369608e-01, 2.69296359e-01, ...,
       1.58512584e-06, 1.58512584e-06, 0.00000000e+00])

In [59]:
# Predicted weights
W_df = pd.DataFrame(W,index = roll_index,columns=['w'])

# Load the original weights from github:
W_original_df = pd.read_csv('weights_validation.csv')

# Set new index, calculate difference between original and predicted:
W_original_df = W_original_df.set_index(W_df.index)
W_original_df['Predicted'] = W_df.w
W_original_df['diff'] = W_original_df.Weight - W_original_df.Predicted

# See where we are off by more than e-6
m = W_original_df.Weight.values - W_df.w.values > 0.000001
W_original_df[m]

Level_id  Agg_Level_1  Agg_Level_2    Weight  Predicted  \
level id                                                                       
1     CA               Level2           CA            X  0.442371   0.442370   
3     HOBBIES          Level4      HOBBIES            X  0.128079   0.128075   
      HOUSEHOLD        Level4    HOUSEHOLD            X  0.303335   0.303330   
4     FOODS_1          Level5      FOODS_1            X  0.062625   0.062623   
      FOODS_2          Level5      FOODS_2            X  0.154642   0.154639   
      HOBBIES_1        Level5    HOBBIES_1            X  0.122088   0.122084   
      HOUSEHOLD_1      Level5  HOUSEHOLD_1            X  0.229594   0.229592   
      HOUSEHOLD_2      Level5  HOUSEHOLD_2            X  0.073741   0.073738   
5     CA_HOBBIES       Level6           CA      HOBBIES  0.058855   0.058852   
      CA_HOUSEHOLD     Level6           CA    HOUSEHOLD  0.142772   0.142769   
      TX_HOUSEHOLD     Level6           TX    HOUSEHOLD  0.086420   0.086419   
      WI_HOBBIES       Level6           WI      HOBBIES  0.027931   0.027930   
6     CA_FOODS_2       Level7           CA      FOODS_2  0.057655   0.057654   
      CA_HOBBIES_1     Level7           CA    HOBBIES_1  0.056463   0.056460   
      CA_HOUSEHOLD_1   Level7           CA  HOUSEHOLD_1  0.104863   0.104862   
      CA_HOUSEHOLD_2   Level7           CA  HOUSEHOLD_2  0.037909   0.037907   
      TX_FOODS_1       Level7           TX      FOODS_1  0.016016   0.016015   
      WI_FOODS_2       Level7           WI      FOODS_2  0.062561   0.062560   
      WI_HOBBIES_1     Level7           WI    HOBBIES_1  0.026375   0.026374   
7     CA_2_HOUSEHOLD   Level8         CA_2    HOUSEHOLD  0.037630   0.037629   
      CA_3_HOUSEHOLD   Level8         CA_3    HOUSEHOLD  0.055870   0.055869   
8     WI_2_FOODS_2     Level9         WI_2      FOODS_2  0.030535   0.030534   
9     FOODS_2_029     Level10  FOODS_2_029            X  0.002862   0.002861   
      FOODS_2_183     Level10  FOODS_2_183            X  0.002679   0.002677   
      FOODS_3_586     Level10  FOODS_3_586            X  0.005074   0.005073   

                          diff  
level id                        
1     CA              0.000002  
3     HOBBIES         0.000004  
      HOUSEHOLD       0.000005  
4     FOODS_1         0.000002  
      FOODS_2         0.000004  
      HOBBIES_1       0.000004  
      HOUSEHOLD_1     0.000002  
      HOUSEHOLD_2     0.000003  
5     CA_HOBBIES      0.000003  
      CA_HOUSEHOLD    0.000004  
      TX_HOUSEHOLD    0.000001  
      WI_HOBBIES      0.000001  
6     CA_FOODS_2      0.000001  
      CA_HOBBIES_1    0.000003  
      CA_HOUSEHOLD_1  0.000002  
      CA_HOUSEHOLD_2  0.000002  
      TX_FOODS_1      0.000001  
      WI_FOODS_2      0.000002  
      WI_HOBBIES_1    0.000001  
7     CA_2_HOUSEHOLD  0.000001  
      CA_3_HOUSEHOLD  0.000001  
8     WI_2_FOODS_2    0.000001  
9     FOODS_2_029     0.000001  
      FOODS_2_183     0.000001  
      FOODS_3_586     0.000001

In [60]:
#PS: As we see our index matches Level_ids and Agg levels of the original dataset, so the csr_matrix works accurately.

In [61]:
SW = W/np.sqrt(S)

In [62]:
sw_df = pd.DataFrame(np.stack((S, W, SW), axis=-1),index = roll_index,columns=['s','w','sw'])
sw_df.to_pickle('sw_df.pkl')

In [63]:
sw_df.head()

s         w        sw
level id                                    
0     all   3.511763e+07  1.000000  0.000169
1     CA    7.341265e+06  0.442370  0.000163
      TX    3.339548e+06  0.269296  0.000147
      WI    3.765354e+06  0.288334  0.000149
2     CA_1  7.499427e+05  0.110888  0.000128

In [64]:
# Function to do quick rollups:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T


# Function to calculate WRMSSE:
def wrmsse(preds, y_true, score_only=False, s = S, w = W, sw=SW):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        return np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw)/12 #<-used to be mistake here
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])/ s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
        return score, score_matrix

In [65]:
# Load S and W weights for WRMSSE calcualtions:
sw_df = pd.read_pickle('sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

# Load roll up matrix to calcualte aggreagates:
roll_mat_df = pd.read_pickle('roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

In [ ]:
valid.drop(['id'], axis=1, inplace=True)

In [73]:
valid.head()

d,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,2.0,1.0,1.0,0.0,4.0,0.0,0.0,4.0,1.0,3.0,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,...,1.0,0.0,14.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
y_true=valid

In [76]:
y_true.head()

d,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,2.0,1.0,1.0,0.0,4.0,0.0,0.0,4.0,1.0,3.0,...,0.0,2.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,...,1.0,0.0,14.0,0.0,1.0,1.0,4.0,0.0,0.0,4.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
valid_pred.drop(['id'], axis=1, inplace=True)

In [78]:
valid_pred.head()

d,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,...,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913
0,0.685397,0.651345,0.689602,0.760516,0.875410,1.078393,0.957094,0.756863,0.703181,0.720692,...,0.888187,1.106833,1.063508,0.759652,0.745267,0.692920,0.768307,0.962939,1.136008,1.070437
1,1.041330,0.976813,0.934107,0.897321,1.049483,1.311205,0.981375,0.362078,0.750064,0.952473,...,0.970673,1.305012,1.454240,0.864558,0.815666,0.787203,0.929613,1.093902,1.246147,1.398081
2,0.702838,0.701076,0.758857,0.830388,0.840845,1.354820,1.667528,0.930712,0.854505,0.897849,...,1.009303,1.420445,1.837298,0.827672,0.758505,0.743554,0.796418,0.919663,1.316452,1.494654
3,0.393409,0.357715,0.376611,0.399401,0.431834,0.451501,0.447883,0.448853,0.404760,0.425879,...,0.392029,0.314133,0.319371,0.362923,0.364867,0.345212,0.404035,0.411054,0.420275,0.418220
4,0.482882,0.470486,0.429711,0.457244,0.504416,0.391476,0.551846,0.416344,0.416020,0.463526,...,0.226207,0.232233,0.258321,0.194951,0.152111,0.062426,0.062026,0.058331,0.072144,0.079430


In [79]:
score = wrmsse(valid_pred, y_true, score_only=True)
score

0.3856547587859749

In [80]:
score1, score_matrix = wrmsse(valid_pred, y_true)
score_df = pd.DataFrame(score_matrix, index = roll_index)
score_df.reset_index(inplace=True)
score_df.head()

,level,id,0,1,2,3,4,5,6,7,...,18,19,20,21,22,23,24,25,26,27
0,0,all,0.025321,0.000453,0.073909,0.065859,0.000617,0.050789,0.002668,0.002484,...,0.061472,0.077882,0.040061,0.002493,0.196527,0.018079,0.000111,0.003608,0.010241,0.000007
1,1,CA,0.036353,0.002667,0.056300,0.046312,0.018461,0.028291,0.000915,0.002573,...,0.015089,0.027373,0.008818,0.007743,0.043977,0.006603,0.000169,0.000593,0.003832,0.000630
2,1,TX,0.000251,0.000049,0.000042,0.005513,0.003075,0.004162,0.000598,0.000228,...,0.002650,0.001085,0.010281,0.000198,0.018709,0.001147,0.000022,0.002562,0.000718,0.000653
3,1,WI,0.002438,0.000449,0.000900,0.002003,0.002104,0.000388,0.002338,0.000167,...,0.002983,0.003838,0.000131,0.002532,0.003793,0.000106,0.000005,0.000413,0.000033,0.000027
4,2,CA_1,0.004831,0.002832,0.004310,0.003369,0.000007,0.001999,0.000578,0.000010,...,0.000592,0.000614,0.000892,0.000350,0.000029,0.000281,0.000101,0.000233,0.000128,0.000311


In [81]:
score1

0.3856547587859749

In [82]:
# next step:

# Improvement should come from:
# bayesian optimization: store_id:tried and did not work because can not based on wrmsse
# Loss function: wrmse: did not work because could not use full data
# Stable CV: by week-year: some store improved and some store did not
# Good features reduction strategy
# Predictions stabilization with NN